In [34]:
import numpy as np
import random

class Structure:
    def __init__(self, seed, seed_next_possible, portion, destination):
        self.history = None
        self.data = None
        self.rect = []
        self.next_possibles = None
        
        self.portion = portion
        self.seed = seed
        self.seed_next_possible = seed_next_possible
        
        self.cleanUp()
        self.destination = destination
        self.preprocess()

        #print("initialized data: \n", self.data)
    
    def preprocess(self):
        start = self.seed_next_possible[0]
        direction = start[-1]
        new_vertex_clean = np.array([start[0], start[1], start[2]])
        self.add_vertex(new_vertex_clean)
        
        span = 4 + int(random.random() * 6 % 6)
        for i in range(span):
            new_vertex_clean = self.get_vertex_forward(new_vertex_clean, direction)
            self.add_vertex(new_vertex_clean)
        
        self.remove_possible(0)
        self.history = np.append(self.history, self.data[1:], axis=0)
        self.rect.append(block_to_rect(self.data, self.portion))
        self.data = np.array([[0,0,0]])
        
        self.to_dest(new_vertex_clean)


    def to_dest(self, new_vertex_clean):
        x_dist = abs(self.destination[0] - new_vertex_clean[0])
        x_start = min(self.destination[0], new_vertex_clean[0])
        startPos = np.array([x_start, new_vertex_clean[1], new_vertex_clean[2]])
        scale = np.array([x_dist+0.07*self.portion, 0.07*self.portion, 0.07*self.portion])
        rect1 = rect(startPos, scale)
        
        y_dist = abs(self.destination[1] - new_vertex_clean[1])
        y_start = min(self.destination[1], new_vertex_clean[1])
        startPos2 = np.array([self.destination[0], y_start, new_vertex_clean[2]])
        scale2 = np.array([0.07,y_dist+0.07*self.portion, 0.07*self.portion])
        rect2 = rect(startPos2, scale2)
        
        self.rect.append(rect1)
        self.rect.append(rect2)
        
        
    def cleanUp(self):
        self.history = self.seed
        self.data = np.array([[0.0,0.0,0.0]])
        self.data = np.append(self.data, self.seed, axis = 0)
        self.rect = []
        self.rect.append(block_to_rect(self.data, self.portion))
        self.data = np.array([[0.0,0.0,0.0]])
        self.next_possibles = self.seed_next_possible

  
    def cost_func(self, pos):
        return random.random()
  
    def add_vertex(self, next_vertex):
        x = next_vertex[0]
        y = next_vertex[1]
        z = next_vertex[2]

        self.data = np.vstack([self.data, np.array([x,y,z])])

    def remove_possible(self, next_index):
        self.next_possibles = np.delete(self.next_possibles, next_index, axis=0)
  
  #define direction:
  #x=-1 -> 0, x=1 -> 1, y=-1 -> 2, y=1 -> 3, z=-1 -> 4, z=1 -> 5,
    def get_next_possible(self, available_contact, direction):
        result = np.array([0,0,0,0])

        available_contact = np.append(available_contact, 0)
        if direction == 0 or direction == 1:
            result = np.vstack([result, available_contact + np.array([0.0,0.07,0.0,3])])
            result = np.vstack([result, available_contact + np.array([0.0,-0.07,0.0,2])])
            result = np.delete(result, 0, axis =0)
            return result

        if direction == 2 or direction == 3:
            result = np.vstack([result, available_contact + np.array([0.07,0.0,0.0,1])])
            result = np.vstack([result, available_contact + np.array([-0.07,0.0,0.0,1])])
            result = np.delete(result, 0, axis =0)
            return result

    
    def get_last_direction(self,direction):
        count =direction[1]*1 + direction[2]*2
        return count
  
    def get_vertex_forward(self, new_vertex_clean, direction):
        if direction == 0:
            return new_vertex_clean + np.array([-0.07,0.0,0.0])

        if direction == 1:
            return new_vertex_clean + np.array([0.07,0.0,0.0])

        if direction == 2:
            return new_vertex_clean + np.array([0.0,-0.07,0.0])

        if direction == 3:
            return new_vertex_clean + np.array([0.0,0.07,0.0])
    
    def generate(self, steps):
        for i in range(steps):
            #print("step ", i+1)
            self.process()
            self.history = np.append(self.history, self.data[1:], axis=0)
            self.rect.append(block_to_rect(self.data, self.portion))
            self.data = np.array([[0,0,0]])



    def process(self):
        #step 1: for all available contacts, assume they will make a turn  
        #print("next_possibles", self.next_possibles)

        #step2: create a 1d array that stores cost for each possible point
        costs = np.array([])
        for next_possible in self.next_possibles:
            costs = np.append(costs, self.cost_func(next_possible))

        #print("costs", costs)

        #step3 find the next_vertex with smallest cost
        next_index = np.argpartition(costs, -1)[-1:]
        next_vertex = self.next_possibles[next_index] 

        #print("next_vertex", next_vertex)

        #step4 add next_vertex to data
        new_direction = next_vertex[0][3]
        #print("new_direction", new_direction)
        new_vertex_clean = np.array([next_vertex[0][0], next_vertex[0][1], next_vertex[0][2]])
        #print("new_vertex_clean", new_vertex_clean)
        self.add_vertex(new_vertex_clean)
        
        span = 3 + int(random.random() * 6 % 6)
        for i in range(span):
            new_vertex_clean = self.get_vertex_forward(new_vertex_clean, new_direction)
            self.add_vertex(new_vertex_clean)

        #print("data: \n", self.data)

        #step5 update next_possible
        self.remove_possible(next_index)
        #print("next_possibles, removed old", self.next_possibles)
        #print("next_vertex", new_vertex_clean)
        #print("new_direction", new_direction)
        new_next_possibles = self.get_next_possible(new_vertex_clean, new_direction)
        self.next_possibles = np.append(self.next_possibles,new_next_possibles, axis=0)
        #print("next_possibles, add new", self.next_possibles)


In [35]:
seed = np.array([[0.0,0.0,0.0], [0.0,0.1,0.0],[0.0,0.2,0.0],[0.0,0.3,0.0], [0.0,0.4,0.0]])
seed_next_possible = np.array([[-0.1,0.4,0.0,0],[0.1,0.4,0.0,1]])


In [36]:
sa = Structure(seed, seed_next_possible, 0.5, np.array([0.4,0.9]))

sa.generate(5)

In [37]:
sa.history

array([[ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.1 ,  0.  ],
       [ 0.  ,  0.2 ,  0.  ],
       [ 0.  ,  0.3 ,  0.  ],
       [ 0.  ,  0.4 ,  0.  ],
       [-0.1 ,  0.4 ,  0.  ],
       [-0.17,  0.4 ,  0.  ],
       [-0.24,  0.4 ,  0.  ],
       [-0.31,  0.4 ,  0.  ],
       [-0.38,  0.4 ,  0.  ],
       [-0.45,  0.4 ,  0.  ],
       [-0.52,  0.4 ,  0.  ],
       [-0.59,  0.4 ,  0.  ],
       [-0.66,  0.4 ,  0.  ],
       [-0.73,  0.4 ,  0.  ],
       [ 0.1 ,  0.4 ,  0.  ],
       [ 0.17,  0.4 ,  0.  ],
       [ 0.24,  0.4 ,  0.  ],
       [ 0.31,  0.4 ,  0.  ],
       [ 0.38,  0.4 ,  0.  ],
       [ 0.45,  0.4 ,  0.  ],
       [ 0.52,  0.4 ,  0.  ],
       [ 0.59,  0.4 ,  0.  ],
       [ 0.59,  0.33,  0.  ],
       [ 0.59,  0.26,  0.  ],
       [ 0.59,  0.19,  0.  ],
       [ 0.59,  0.12,  0.  ],
       [ 0.59,  0.47,  0.  ],
       [ 0.59,  0.54,  0.  ],
       [ 0.59,  0.61,  0.  ],
       [ 0.59,  0.68,  0.  ],
       [ 0.59,  0.75,  0.  ],
       [ 0.59,  0.82,  0.  ],
       [ 0

In [38]:
seed2 = np.array([[0.0,0.0,0.0], [0.1,0.0,0.0],[0.2,0.0,0.0],[0.3,0.0,0.0], [0.4,0.0,0.0]])
seed2_next_possible = np.array([[0.4,0.1,0.0,3],[0.4,-0.1,0.0,2]])
sa2 = Structure(seed2, seed2_next_possible, 0.5,np.array([0.4,0.9]))

sa2.generate(5)

In [39]:
sa.history = np.round(sa.history, 1)

In [40]:
sa.history

array([[ 0. ,  0. ,  0. ],
       [ 0. ,  0.1,  0. ],
       [ 0. ,  0.2,  0. ],
       [ 0. ,  0.3,  0. ],
       [ 0. ,  0.4,  0. ],
       [-0.1,  0.4,  0. ],
       [-0.2,  0.4,  0. ],
       [-0.2,  0.4,  0. ],
       [-0.3,  0.4,  0. ],
       [-0.4,  0.4,  0. ],
       [-0.5,  0.4,  0. ],
       [-0.5,  0.4,  0. ],
       [-0.6,  0.4,  0. ],
       [-0.7,  0.4,  0. ],
       [-0.7,  0.4,  0. ],
       [ 0.1,  0.4,  0. ],
       [ 0.2,  0.4,  0. ],
       [ 0.2,  0.4,  0. ],
       [ 0.3,  0.4,  0. ],
       [ 0.4,  0.4,  0. ],
       [ 0.5,  0.4,  0. ],
       [ 0.5,  0.4,  0. ],
       [ 0.6,  0.4,  0. ],
       [ 0.6,  0.3,  0. ],
       [ 0.6,  0.3,  0. ],
       [ 0.6,  0.2,  0. ],
       [ 0.6,  0.1,  0. ],
       [ 0.6,  0.5,  0. ],
       [ 0.6,  0.5,  0. ],
       [ 0.6,  0.6,  0. ],
       [ 0.6,  0.7,  0. ],
       [ 0.6,  0.8,  0. ],
       [ 0.6,  0.8,  0. ],
       [ 0.6,  0.9,  0. ],
       [ 0.6,  1. ,  0. ],
       [ 0.6,  1. ,  0. ],
       [ 0.7,  1. ,  0. ],
 

In [41]:
sa2.history

array([[ 0.  ,  0.  ,  0.  ],
       [ 0.1 ,  0.  ,  0.  ],
       [ 0.2 ,  0.  ,  0.  ],
       [ 0.3 ,  0.  ,  0.  ],
       [ 0.4 ,  0.  ,  0.  ],
       [ 0.4 ,  0.1 ,  0.  ],
       [ 0.4 ,  0.17,  0.  ],
       [ 0.4 ,  0.24,  0.  ],
       [ 0.4 ,  0.31,  0.  ],
       [ 0.4 ,  0.38,  0.  ],
       [ 0.4 ,  0.45,  0.  ],
       [ 0.4 ,  0.52,  0.  ],
       [ 0.4 , -0.1 ,  0.  ],
       [ 0.4 , -0.17,  0.  ],
       [ 0.4 , -0.24,  0.  ],
       [ 0.4 , -0.31,  0.  ],
       [ 0.4 , -0.38,  0.  ],
       [ 0.4 , -0.45,  0.  ],
       [ 0.4 , -0.52,  0.  ],
       [ 0.4 , -0.59,  0.  ],
       [ 0.4 , -0.66,  0.  ],
       [ 0.47, -0.66,  0.  ],
       [ 0.54, -0.66,  0.  ],
       [ 0.61, -0.66,  0.  ],
       [ 0.68, -0.66,  0.  ],
       [ 0.68, -0.59,  0.  ],
       [ 0.68, -0.52,  0.  ],
       [ 0.68, -0.45,  0.  ],
       [ 0.68, -0.38,  0.  ],
       [ 0.68, -0.31,  0.  ],
       [ 0.68, -0.24,  0.  ],
       [ 0.68, -0.17,  0.  ],
       [ 0.68, -0.1 ,  0.  ],
       [ 0

In [42]:
def offscreen(structure, max_coordinate, min_coordinate):
    max_x = max_coordinate[0]
    max_y = max_coordinate[1]
    min_x = min_coordinate[0]
    min_y = min_coordinate[1]
    
    for i in structure.rect:
        if(i.start_x < min_x or i.start_y < min_y):
            return True
        if(i.start_x+i.scale_x > max_x or i.start_y+i.scale_y > max_y):
            return True
    
    return False

In [43]:
max_coordinate = np.array([1.0, 1.0])
min_coordinate = np.array([-1.0, -1.0])
offscreen(sa, max_coordinate, min_coordinate)

True

In [44]:
kk = []

In [45]:
kk.append(sa)

In [46]:
kk[0].rect

In [47]:
for i in sa.rect:
    i.info()

start_x: 0.0 start_y: 0.0 start_z: 0.0
scale_x: 0.05 scale_y: 0.45 scale_z: 0.05
 
start_x: -0.73 start_y: 0.4 start_z: 0.0
scale_x: 0.68 scale_y: 0.05 scale_z: 0.05
 
start_x: -0.73 start_y: 0.4 start_z: 0.0
scale_x: 1.17 scale_y: 0.04 scale_z: 0.04
 
start_x: 0.4 start_y: 0.4 start_z: 0.0
scale_x: 0.07 scale_y: 0.54 scale_z: 0.04
 
start_x: 0.1 start_y: 0.4 start_z: 0.0
scale_x: 0.54 scale_y: 0.05 scale_z: 0.05
 
start_x: 0.59 start_y: 0.12 start_z: 0.0
scale_x: 0.05 scale_y: 0.26 scale_z: 0.05
 
start_x: 0.59 start_y: 0.47 start_z: 0.0
scale_x: 0.05 scale_y: 0.61 scale_z: 0.05
 
start_x: 0.66 start_y: 1.03 start_z: 0.0
scale_x: 0.54 scale_y: 0.05 scale_z: 0.05
 
start_x: 0.66 start_y: 0.12 start_z: 0.0
scale_x: 0.33 scale_y: 0.05 scale_z: 0.05
 


In [48]:
def parallel_score(history_a, history_b):
    score = 0
    parallel_pts = []
    
    for i in history_a:
        for j in history_b:
            if(i[0] == j[0] and i[1] == j[1]):
                parallel_pts.append((i[0],i[1]))
                score += 5
            elif(i[0] == j[0] or i[1] == j[1]):
                score += 1
    return score, parallel_pts

In [49]:
def check_close(struct_a):
    for i in struct_a.history:
        print(i)

In [50]:
def occlude(front_structure,position, eye):
    ray = eye - position
    ray = ray / np.linalg.norm(ray)
#     print("ray", ray)

    z_depth = front_structure.history[0][-1]
    t = (eye[-1] - z_depth) / ray[-1]
#     print("t", t)

    x = eye[0] + t*ray[0]
    y = eye[1] + t*ray[1]
    
    for i in front_structure.history:
        if x== i[0] and y == i[1]:
            return True
    
    return False

In [51]:
def occlusion_score(front_structure, points, eye):
    
    count = 0
    for i in points:
        if (occlude(front_structure, i, eye)):
            count += 1
    
    return count
        
    

IndentationError: expected an indented block (482663588.py, line 2)

In [69]:
round(random.uniform(1, 6)/100,2)

0.02

In [52]:
eye = np.array([5.0,5.0,5.0])

In [53]:
position = np.array([0.3,0.4,-0.25])

In [54]:
occlude(sa, position, eye)

False

In [20]:
class connecting_structure:
    def __init__(self, x, y, foreground_z, background_z):
        self.x = x
        self.y = y
        self.foreground_z = foreground_z
        self.background_z = background_z
    
    def get_object():
        z_start = min(self.foreground_z, self.background_z)
        z_scale = abs(self.foreground_z - self.background_z )
        
        startPos = np.array([x,y,z_start])
        scale = np.array([0.07, 0.07, z_scale])
        
        tempt = rect(startPos, scale)
        
        return tempt
    
    def get_center():
        z_start = min(self.foreground_z, self.background_z)
        z_scale = abs(self.foreground_z - self.background_z )
        
        center = np.array([self.x+0.035, self.y+0.035, z_start + 0.5*z_scale])
        
        return center

In [21]:
sa.history[0][-1]

NameError: name 'sa' is not defined

In [22]:
a = round(int(random.random() * 5 % 5) * 0.1 + 0.2, 1)

In [23]:
b = random.random()

In [24]:
b>0.5 ? -1 : 1

SyntaxError: invalid syntax (202455723.py, line 1)

In [25]:
check_close(sa2)

NameError: name 'sa2' is not defined

In [26]:
(score, parallel_pts)= parallel_score(sa2.history, sa.history)
score

NameError: name 'sa2' is not defined

In [27]:
len(parallel_pts)

NameError: name 'parallel_pts' is not defined

In [28]:
def block_to_rect(buffer, portion):
    start = buffer[1]
    end = buffer[-1]
    x_start = min(start[0],end[0])
    y_start = min(start[1],end[1])    
    z_start = min(start[2],end[2])
    
    x_scale = max(abs(start[0] - end[0]), 0.1 * portion)
    y_scale = max(abs(start[1] - end[1]), 0.1 * portion)
    z_scale = max(abs(start[2] - end[2]), 0.1 * portion)
    
    if(x_scale > 0.1 * portion):
        x_scale += 0.1 * portion
        
    if(y_scale > 0.1 * portion):
        y_scale += 0.1 * portion
        
    if(z_scale > 0.1 * portion):
        z_scale += 0.1 * portion

    startPos = np.array([x_start,y_start,z_start])
    scale = np.array([x_scale,y_scale,z_scale])
    
    tempt = rect(startPos, scale)
    return tempt


In [29]:
class rect:
    def __init__(self, startPos, scale): 
        self.start_x = round(startPos[0],2);
        self.start_y = round(startPos[1],2);
        self.start_z = round(startPos[2],2);
        
        self.scale_x = round(scale[0],2);
        self.scale_y = round(scale[1],2);
        self.scale_z = round(scale[2],2);
    
    def info(self):
        print("start_x:",self.start_x,"start_y:",self.start_y,"start_z:",self.start_z)
        print("scale_x:",self.scale_x,"scale_y:",self.scale_y,"scale_z:",self.scale_z)
        print(" ")
    
    def center_x(self):
        return round(self.start_x + 0.5 * self.scale_x, 1)
    
    def center_y(self):
        return round(self.start_y + 0.5 * self.scale_y,1)

In [30]:
b1 = np.array([[0,0,0], [0,1,0],[0,2,0],[0,3,0], [0,4,0]])
b2 = np.array([[0,0,1], [0,1,1],[0,2,1],[0,3,1], [0,4,1]])

In [31]:
c = np.append(b1, b2, axis=0)
c

array([[0, 0, 0],
       [0, 1, 0],
       [0, 2, 0],
       [0, 3, 0],
       [0, 4, 0],
       [0, 0, 1],
       [0, 1, 1],
       [0, 2, 1],
       [0, 3, 1],
       [0, 4, 1]])

In [32]:
rect_list = []

In [33]:
rect_list.append(block_to_rect(b1))
rect_list.append(block_to_rect(b2))
rect_list

TypeError: block_to_rect() missing 1 required positional argument: 'portion'

In [187]:
pos = np.array([2.0,2.0,4.0])
np.array([[0.0,0.0,0.0]])

array([[0., 0., 0.]])

In [202]:
len(sa.next_possibles)

6

In [192]:
import random
def rand_index(input_array):
    return int(random.random()* len(input_array) % len(input_array))

In [206]:
for i in range(20):
    print(rand_index(sa.next_possibles))

0
1
5
1
5
2
1
3
5
5
5
4
0
0
5
1
5
2
0
2


In [38]:
pos[0]

2.0

In [45]:
def gen_seed(pos):
    seed = pos
    for i in range(1,5):
        tempt = np.array([pos[0]+0.1*i,pos[1],pos[2]])
        seed = np.vstack([seed, tempt])
    return seed

In [46]:
a=gen_seed(pos)

In [56]:
a[-1][0]

2.4

In [51]:
def get_next_possible(pos):
    tempt1 = np.array([pos[0],pos[1]+0.1,pos[2],4])
    tempt2 = np.array([pos[0],pos[1]-0.1,pos[2],3])
    
    return np.vstack([tempt1, tempt2])

In [52]:
pos_b = a[-1]

In [53]:
x = get_next_possible(pos_b)
x

array([[2.4, 2.1, 4. , 4. ],
       [2.4, 1.9, 4. , 3. ]])

In [218]:
def too_close(structure):
    horizontal = []
    vertical = []

    for i in structure.rect:
        if(i.scale_x > i.scale_y):
            horizontal.append(i)
        else:
            vertical.append(i)
    
    for i in horizontal:
        for j in horizontal:
            if(i.start_y != j.start_y and abs(i.start_y - j.start_y)<0.2):
                return True
    
    for i in vertical:
        for j in vertical:
            if(i.start_x != j.start_x and abs(i.start_x - j.start_x)<0.2):
                return True
    
    return False

In [219]:
too_close(sa)

True

In [210]:
sa.rect

In [223]:
import openai

response = openai.Image.create(
  prompt="a white siamese cat",
  n=1,
  size="1024x1024"
)
image_url = response['data'][0]['url']


AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://onboard.openai.com for details, or email support@openai.com if you have any questions.